### loading modules and data

In [1]:
########################################################

# If running with Google Colab

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# # If using colab
# from google.colab import files
# files.upload()
# # select the 3 .py files (models, utils, data_utils)

In [3]:
########################################################

# If running with Google Colab
# Create a folder "C247" and then store the project datasets within that folder
# Check that your datasets are setup correctly

!ls "/content/gdrive/My Drive/C247" # File path

EEG_loading.ipynb	PyTorch_Experiments.ipynb  X_train_valid.npy
person_test.npy		pytorch_rnn_example.ipynb  y_test.npy
person_train_valid.npy	X_test.npy		   y_train_valid.npy


In [4]:
from models import *
from utils import *
from data_utils import *

GPU is available


In [5]:
# X_test = np.load("X_test.npy")
# y_test = np.load("y_test.npy")
# person_train_valid = np.load("person_train_valid.npy")
# X_train_valid = np.load("X_train_valid.npy")
# y_train_valid = np.load("y_train_valid.npy")
# person_test = np.load("person_test.npy")

# Change if your directory is different

# dataset_path = './data/' # Yiming Path
dataset_path = "/content/gdrive/My Drive/C247/" 

X_test = np.load(dataset_path + "X_test.npy")
y_test = np.load(dataset_path + "y_test.npy")
person_train_valid = np.load(dataset_path + "person_train_valid.npy")
X_train_valid = np.load(dataset_path + "X_train_valid.npy")
y_train_valid = np.load(dataset_path + "y_train_valid.npy")
person_test = np.load(dataset_path + "person_test.npy")
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [6]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.4
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

(8460, 22, 1000)

### K-Fold Training and Validation (use k=1 to get results faster)

In [7]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 20

for k in range(num_folds):
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=0.5, weight_decay=0)
    print ('fold {}'.format(k+1))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
    X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]

    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
    if aug_type != 'window':
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


RNN TYPE: GRU
WEIGHT DECAY: 0
LEARNING RATE: 0.001
fold 1
epoch: 1      time: 32.05    loss: 1461.974    train acc: 0.277    val acc: 0.313
saving best model...
epoch: 2      time: 31.87    loss: 1400.384    train acc: 0.360    val acc: 0.415
saving best model...
epoch: 3      time: 31.92    loss: 1314.646    train acc: 0.428    val acc: 0.428
saving best model...
epoch: 4      time: 31.81    loss: 1253.591    train acc: 0.464    val acc: 0.441
saving best model...
epoch: 5      time: 31.86    loss: 1200.903    train acc: 0.495    val acc: 0.455
saving best model...
epoch: 6      time: 31.86    loss: 1153.805    train acc: 0.525    val acc: 0.427
epoch: 7      time: 31.73    loss: 1110.656    train acc: 0.548    val acc: 0.406
epoch: 8      time: 31.98    loss: 1071.040    train acc: 0.568    val acc: 0.427
epoch: 9      time: 31.81    loss: 1040.244    train acc: 0.583    val acc: 0.414
epoch: 10     time: 31.90    loss: 1010.570    train acc: 0.598    val acc: 0.409
epoch: 11     tim

### Training and Testing

In [8]:
# Uncomment if using cwt
X_test, y_test, person_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_test, y_test, person_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.4916
